In [16]:
import time
import datetime
import pandas as pd
import requests
from io import BytesIO

In [48]:
def date_to_timestamp(date,utc=False):
    """
    str형태의 date를 timestamp로 만들어주기
    :params (str or datetime) date : '%Y-%m-%d %H:%M:%S'형태의 데이터. ex)'2023-01-18 23:00:00'
    :params bool utc : True로 설정할시에 date를 utc 시간이라고 생각
    :return timestamp시간(단위 ms) ex)1674050400000
    :rtype int
    
    ex) date_to_timestamp('2023-01-18 23:00:00') -> 1674050400000
    """
    if type(date) == str: # str인 경우 datetime으로 변환해주기
        dt = datetime.datetime.strptime(date,'%Y-%m-%d %H:%M:%S')
    else: # datetime으로 들어온경우
        dt = date
        
    # time.mktime은 local 타임 기준으로 timestamp를 변경함. 따라서 utc일 경우 +9시간해서 한국시간으로 설정해줘야함
    if utc:
        dt = date + datetime.timedelta(hours = 9)
    
    ts = time.mktime(dt.timetuple()) 
    
    return int(ts*1000)

now_timestamp = date_to_timestamp('2023-01-30 20:14:00')
print(now_timestamp)

1675077240000


In [63]:
url = f'https://www.binance.com/api/v3/uiKlines?limit=1000&symbol=BTCUSDT&interval=1s&startTime={now_timestamp}'
webpage = requests.get(url)

df = pd.read_json(BytesIO(webpage.content))
df = df[[0,1,2,3,4,5]]
df.rename(columns={0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'},inplace=True)
df['datetime']

0      1675077240000
1      1675077241000
2      1675077242000
3      1675077243000
4      1675077244000
           ...      
995    1675078235000
996    1675078236000
997    1675078237000
998    1675078238000
999    1675078239000
Name: datetime, Length: 1000, dtype: int64

In [50]:
import datetime
import pytz

# timestamp -> datetime으로 변경하기
df[0] = pd.to_datetime(df[0], unit='ms') + datetime.timedelta(hours = 9)
df

,0,1,2,3,4,5
0,2023-01-30 20:14:00,23247.13,23247.52,23245.20,23246.87,2.29732
1,2023-01-30 20:14:01,23246.85,23247.52,23246.15,23247.10,1.97894
2,2023-01-30 20:14:02,23247.10,23247.52,23246.85,23247.43,1.00228
3,2023-01-30 20:14:03,23247.52,23247.52,23246.15,23246.29,0.46596
4,2023-01-30 20:14:04,23246.81,23247.15,23245.98,23246.81,2.29022
...,...,...,...,...,...,...
995,2023-01-30 20:30:35,23255.32,23256.09,23254.78,23255.01,0.88726
996,2023-01-30 20:30:36,23254.74,23257.98,23254.74,23256.29,7.29588
997,2023-01-30 20:30:37,23256.36,23257.28,23254.50,23254.66,2.88746
998,2023-01-30 20:30:38,23254.67,23255.21,23254.15,23255.21,2.00879


In [88]:
def make_second_csv_data(coin_name,start_time,end_time):
    """
    coin이름, 시작, 끝 시간을 지정해주면 그 기간까지의 1s 데이터를 수집하여 csv 파일로 반환
    :params str coin_name : 코인이름 ex)"BTCUSDT"
    :params str start_time : 수집 시작 시간 ex) '2022-01-01 00:00:00' (한국시간기준)
    :params str end_time : 수집 끝 시간 ex) '2023-01-01 00:00:00' (한국시간기준)
    :return None
    :rtype None
    
    f'{coin_name}_1s_{start_time}_{end_time}.csv' 파일로 저장됨
    """
    
    
    start_timestamp = date_to_timestamp(start_time) # str -> timestamp
    
    end_time = datetime.datetime.strptime(end_time,'%Y-%m-%d %H:%M:%S') # str -> datetime
    
    url = f'https://www.binance.com/api/v3/uiKlines?limit=1000&symbol={coin_name}&interval=1s&startTime={start_timestamp}'
    webpage = requests.get(url)

    df = pd.read_json(BytesIO(webpage.content))
    df = df[[0,1,2,3,4,5]]
    df.rename(columns={0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'},inplace=True)
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') + datetime.timedelta(hours = 9)
    df.set_index('datetime', inplace=True)
    
    while len(df)==0:  # 정기 점검이 있는 시간대에는 조회를 해도 결과가 나오지 않음. 1000개씩 조회되므로 데이터가 조회될때까지 500개씩 건너뛰기
        
        start_timestamp=date_to_timestamp(datetime.datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(senconds=1000))
    
        url = f'https://www.binance.com/api/v3/uiKlines?limit=1000&symbol={coin_name}&interval=1s&startTime={start_timestamp}'
        webpage = requests.get(url)

        df = pd.read_json(BytesIO(webpage.content))
        df = df[[0,1,2,3,4,5]]
        df.rename(columns={0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'},inplace=True)
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') + datetime.timedelta(hours = 9)
        df.set_index('datetime', inplace=True)
    total_df = df
    
    check_count = 0
    
    while True:
        
        check_count+=1
        if check_count%10 == 0:
            print(total_df.index[-1])
        
        if end_time <= df.index[-1]:
            break
        
        time_later = df.index[-1] + datetime.timedelta(seconds=1000)
        
        time_later = date_to_timestamp(time_later)
        url = f'https://www.binance.com/api/v3/uiKlines?limit=1000&symbol={coin_name}&interval=1s&startTime={time_later}'
        webpage = requests.get(url)

        df = pd.read_json(BytesIO(webpage.content))
        df = df[[0,1,2,3,4,5]]
        df.rename(columns={0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'},inplace=True)
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') + datetime.timedelta(hours = 9)
        df.set_index('datetime', inplace=True)
        
        while len(df)==0:  # 정기 점검이 있는 시간대에는 조회를 해도 결과가 나오지 않음. 1000개씩 조회되므로 데이터가 조회될때까지 500개씩 건너뛰기
            time_later=datetime.datetime.strptime(time_later,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(senconds=1000) 
            time_later = date_to_timestamp(time_later)
            url = f'https://www.binance.com/api/v3/uiKlines?limit=1000&symbol={coin_name}&interval=1s&startTime={time_later}'
            webpage = requests.get(url)

            df = pd.read_json(BytesIO(webpage.content))
            df = df[[0,1,2,3,4,5]]
            df.rename(columns={0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'},inplace=True)
            df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') + datetime.timedelta(hours = 9)
            df.set_index('datetime', inplace=True)
            
        total_df = pd.concat([total_df,df])
        
        time.sleep(1)
    
    total_df = total_df[:end_time]
    
    coin_name = "".join(coin_name.split("/"))
    s_time = "-".join("-".join(str(start_time).split(" ")).split(":"))
    e_time = "_".join("-".join(str(end_time).split(" ")).split(":"))
    total_df.to_csv(f'./{coin_name}_1s_{s_time}_{e_time}.csv')

In [89]:
make_second_csv_data('BTCUSDT','2023-01-29 23:00:00','2023-01-30 00:00:00')